<a href="https://colab.research.google.com/github/PedroTonus/praticasGSI073/blob/main/GSI073_aula0_support_vector_machine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GSI073 - Tópicos Especiais de Inteligência Artificial

Neste notebook, um tipo de Support Vector Machine Linear.


## Preparação dos dados

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn import datasets
from sklearn.model_selection import train_test_split

# Preparar o dataset
iris = datasets.load_iris()
X = iris.data; y = iris.target

X = X[y != 1] ; y = y[y != 1] # Remove a classe 'versicolor' (classe 1)
y = torch.tensor(y, dtype=torch.float32)
y[y == 0] = -1  # SVM espera rótulos em {-1, +1} para a primeira classe
y[y == 2] = 1   # Mapeia a segunda classe (originalmente 2) para +1

X = torch.tensor(X, dtype=torch.float32) # Tensor é um tipo especial que suporta muitas dimensões

# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

A nossa Support Vector Machine é basicamente um hiperplano definido por *w* e *b* que melhor separa as classes.

In [ ]:
# Definir parâmetros treináveis da Support Vector Machine: w e b
n_features = X_train.shape[1]
w = torch.randn(n_features, 1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# === Hiperparâmetros ===
learning_rate = 0.01
epochs = 300
optimizer = optim.Adam([w, b], lr=learning_rate)

## Execução do treinamento

In [ ]:
for epoch in range(epochs):
    optimizer.zero_grad()

    y_pred = X_train @ w + b  # Modelo SVM (um hiperplano que depende de w e b) treinado com dados de treino

    # Hinge loss: max(0, 1 - y_i * (w^T x_i + b))
    perda_de_classificacao = torch.clamp(1 - y_train.view(-1, 1) * y_pred, min=0).mean()

    # Termo de regularização
    perda_de_distancia_entre_classes = 0.5 * torch.sum(w ** 2) # 2/||w|| é a distância que queremos que seja a maior possível

    # Função objetivo tradicional: minimizar reg + C * hinge
    loss = perda_de_distancia_entre_classes + perda_de_classificacao

    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss={loss.item():.4f}")

# Compare o resultado da Support Vector Machine com o resultado da regressão logística

A SVM (Support Vector Machine) procura uma linha (ou plano) que separe as classes de dados — por exemplo, tipos de flores — com a maior distância possível entre elas. Essa distância é chamada de margem.
Quando os dados podem ser separados perfeitamente por uma linha reta (ou plano), a SVM consegue encontrar essa divisão perfeita, o que resulta em 100% de acerto.

Já a Regressão Logística tenta calcular a probabilidade de cada ponto pertencer a uma classe, usando uma curva sigmoide. Ela também cria uma linha de separação, mas o seu foco é ajustar as probabilidades aos dados observados, e não necessariamente maximizar a margem entre as classes.
Por isso, mesmo quando os dados podem ser separados perfeitamente, a Regressão Logística pode não alcançar 100% de acerto, especialmente se houver pontos muito próximos da linha de separação ou se o processo de ajuste não for perfeito